# Tree-based Ultrametric Measures: Patient Comparison

Compute tree-based distances (Robinson-Foulds, Cophenetic correlation, Baker's gamma, Fowlkes-Mallows) across phases for every patient, per EEG band.
Plots show, for each band and measure, phase-by-phase distance matrices side-by-side for all patients.


In [ ]:
%matplotlib inline
# Core imports
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Project helpers
from lrgsglib import *
move_to_rootf(pathname='lrg_eegfc')
from lrg_eegfc import *

# Specific utilities
from lrgsglib.utils.lrg.infocomm import extract_ultrametric_matrix
from lrg_eegfc.utils.corrmat import process_network_for_phase
from lrg_eegfc.utils.datamanag.loaders import load_data_dict
from lrg_eegfc.config.const import PHASE_LABELS, BRAIN_BANDS, PATIENTS_LIST, BRAIN_BAND_LABELS

# Figure output path
path_figs = Path('data') / 'figures' / 'new_tree_measures_pat_comparison'
path_figs.mkdir(parents=True, exist_ok=True)

phase_labels = list(PHASE_LABELS)
bands = list(BRAIN_BANDS.keys())
patients = list(PATIENTS_LIST)

# Correlation network protocol (consistent with other notebooks)
correlation_protocol = dict(threshold=0, spectral_cleaning=False)

print(f'Patients: {patients}')
print(f'Phases: {phase_labels}')
print(f'Bands: {bands}')

from lrgsglib.utils.basic.linalg import (
    tree_robinson_foulds_distance,
    tree_cophenetic_correlation,
    tree_baker_gamma,
    tree_fowlkes_mallows_index,
)


## Load Data

In [ ]:
data_dict, int_label_map = load_data_dict()
print('✓ Data loaded')


## Build Ultrametric and Linkage Structures per Patient, Phase, Band

In [ ]:
def compute_structures_for_patient(patient: str, filter_order: int = 1):
    U = {b: {} for b in bands}
    Z = {b: {} for b in bands}
    D = {b: {} for b in bands}

    pin_labels = int_label_map[patient]['label']

    for phase in phase_labels:
        data_pat_phase = data_dict[patient][phase]
        data_pat_phase_ts = data_pat_phase['data']
        fs_raw = data_pat_phase.get('fs', None)
        fs = float(np.asarray(fs_raw).flat[0]) if fs_raw is not None else 1000.0

        for band in bands:
            G, label_dict, lnkgM, clTh, corr_mat, dists = process_network_for_phase(
                data_pat_phase_ts, fs, band, correlation_protocol, pin_labels, filter_order=filter_order
            )
            if lnkgM is None or G is None:
                U[band][phase] = None
                Z[band][phase] = None
                D[band][phase] = None
                continue
            try:
                U[band][phase] = extract_ultrametric_matrix(lnkgM, G.number_of_nodes())
            except Exception as e:
                print(f'[WARN] {patient} {phase} {band}: failed to extract ultrametric ({e})')
                U[band][phase] = None
            Z[band][phase] = lnkgM
            if isinstance(dists, dict):
                D[band][phase] = dists.get('condensed')
            else:
                D[band][phase] = None
    return U, Z, D

ultra_by_pat = {}
linkage_by_pat = {}
condensed_by_pat = {}
for pat in patients:
    print(f'Computing ultrametric structures for {pat} ...')
    U, Z, D = compute_structures_for_patient(pat)
    ultra_by_pat[pat] = U
    linkage_by_pat[pat] = Z
    condensed_by_pat[pat] = D
print('✓ Ultrametric structures computed for all patients')


## Phase Distance Matrices per Patient, Band, Measure

In [ ]:
measures = ['robinson_foulds', 'cophenetic', 'baker_gamma', 'fowlkes_mallows']
results = {m: {b: {} for b in bands} for m in measures}
n = len(phase_labels)

for band in bands:
    for pat in patients:
        for measure in measures:
            dm = np.full((n, n), np.nan, dtype=float)
            for i, pi in enumerate(phase_labels):
                for j, pj in enumerate(phase_labels):
                    if i == j:
                        dm[i, j] = 0.0
                        continue
                    Z1 = linkage_by_pat[pat][band].get(pi)
                    Z2 = linkage_by_pat[pat][band].get(pj)
                    if Z1 is None or Z2 is None:
                        dm[i, j] = np.nan
                    else:
                        if measure == 'robinson_foulds':
                            dm[i, j] = tree_robinson_foulds_distance(Z1, Z2, normalized=True)
                        elif measure == 'cophenetic':
                            corr = tree_cophenetic_correlation(Z1, Z2)
                            dm[i, j] = 1.0 - corr
                        elif measure == 'baker_gamma':
                            gamma = tree_baker_gamma(Z1, Z2)
                            dm[i, j] = 1.0 - gamma
                        elif measure == 'fowlkes_mallows':
                            fm = tree_fowlkes_mallows_index(Z1, Z2)
                            dm[i, j] = 1.0 - fm
            results[measure][band][pat] = dm
    print(f'Band {band}: ✓ measures computed')

print('✓ All phase-distance matrices computed')


## Plot: Side-by-Side Patients per Band and Measure

In [ ]:
def plot_band_measure_side_by_side(measure: str, band: str, label: str):
    vmax = 0.0
    for pat in patients:
        m = results[measure][band].get(pat)
        if m is not None and np.isfinite(m).any():
            vmax = max(vmax, np.nanmax(m))
    if vmax == 0.0:
        print(f'[SKIP] No finite values for {measure} / {band}')
        return

    fig, axes = plt.subplots(1, len(patients), figsize=(4*len(patients)+2, 4), constrained_layout=True)
    if len(patients) == 1:
        axes = [axes]

    cmap = plt.cm.viridis.copy()
    cmap.set_bad(color='lightgray')

    for ax, pat in zip(axes, patients):
        M = results[measure][band][pat]
        im = ax.imshow(M, vmin=0.0, vmax=vmax, cmap=cmap, aspect='equal')
        ax.set_title(f"{pat}")
        ax.set_xticks(range(len(phase_labels)))
        ax.set_yticks(range(len(phase_labels)))
        ax.set_xticklabels(phase_labels, rotation=45, ha='right')
        ax.set_yticklabels(phase_labels)
        for spine in ax.spines.values():
            spine.set_visible(False)

    cbar = fig.colorbar(im, ax=axes, shrink=0.85)
    cbar.set_label(label)
    outdir = path_figs / measure
    outdir.mkdir(parents=True, exist_ok=True)
    outfile = outdir / f'{band}.png'
    fig.savefig(outfile, dpi=200, bbox_inches='tight')
    plt.show()
    print(f'Saved: {outfile}')

labels = {
    'robinson_foulds': 'Robinson-Foulds distance',
    'cophenetic': '1 - cophenetic correlation',
    'baker_gamma': "1 - Baker's gamma",
    'fowlkes_mallows': '1 - Fowlkes-Mallows index',
}

for measure in measures:
    for band in bands:
        plot_band_measure_side_by_side(measure, band, labels[measure])

print('✓ All figures generated')
